<a href="https://colab.research.google.com/github/ajrauthan/BA775-Analyzing-F1-Racing-performance/blob/main/BA820_Team_Project_Market%20Basket%20Analysis(Alternative%20Approach).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing libraries and loading the dataset

In [ ]:
#importing necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# loading all dataframes

aisles = pd.read_csv('/content/aisles.csv')
print(aisles.shape)
dept = pd.read_csv('/content/departments.csv')
print(dept.shape)
orders_prior = pd.read_csv('/content/order_products__prior.csv')
print(orders_prior.shape)
orders = pd.read_csv('/content/orders.csv')
print(orders.shape)
prod = pd.read_csv('/content/products.csv')
print(prod.shape)

(134, 2)
(21, 2)
(32434489, 4)
(3421083, 7)
(49688, 4)


In [ ]:
# joining them together on common keys

df1 = pd.merge(prod, aisles, how='inner', on='aisle_id')
df2 = df1.merge(dept, how='inner', on='department_id')
df3 = df2.merge(orders_prior, how='inner', on='product_id')
df4 = df3.merge(orders, how='inner', on='order_id')

df = df4.drop(['aisle_id','department_id','eval_set','order_number'],axis=1)
display(df.head())
#simplifying pnenomics and displaying the dataframe
print(df.shape[1], df.shape[0])

##Exploring the dataset

In [ ]:
# df.eval_set.unique()

In [ ]:
# df = df.drop('eval_set', axis=1)

Dropping the eval_set column since it holds no relevance and is merely a label for orders with only one label present.

In [ ]:
df.isna().sum()

Null values are only observed in the 'days_since_prior_order' column. We shall explore this further now.

In [ ]:
df[df.days_since_prior_order.isnull()]

These null values might be suggesting that no prior purchase of the product was made by the customer. We can impute it with an arbitrary float value instead so that it is in line with the datatype of the column and holds value.

In [ ]:
# imputing a value to null values of days_since_prior_order so that those rows can be used for meaningful analysis

df['days_since_prior_order'].fillna(9999.0, inplace=True) # here 9999 is our arbitrary number
df['days_since_prior_order'].isna().sum()

In [ ]:
df.info()

The data types seem relevant to their respective columns.

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

Having a look at the summary statistics gives us a clearer picture of the dataset to perform further analysis.

In [ ]:
# converting day of the week (dow) of order from numerical to weekday name

# Define a mapping dictionary
dow_mapping = {
    0: 'Sunday',
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday'
}

# Use the map function to apply the mapping
df['order_dow'] = df['order_dow'].map(dow_mapping)
df

In [ ]:
df.info()

In [ ]:
df.isna().sum()

##EDA

Performing various exploratory data analyses to get to know the data better.

In [ ]:
# number of unique products in each aisle

df.groupby('aisle').product_id.nunique()

In [ ]:
# number of unique products in each departments

df.groupby('department').product_id.nunique()

In [ ]:
# order id with the biggest order

max_order = df.add_to_cart_order.max()
df[df.add_to_cart_order==max_order].loc[:,['order_id', 'user_id']]

In [ ]:
# top customers based on purchase quantity

top_cust = df.groupby('user_id').order_id.count().reset_index()
top_cust.sort_values('order_id', ascending = False).head(10)

In [ ]:
# most popular products based on purchases

top_prod = df.product_name.value_counts().reset_index()
top_prod = top_prod.sort_values('product_name', ascending = False).head(10)

sns.barplot(data=top_prod, x='index', y='product_name')
plt.xlabel('Product Name')
plt.ylabel('Quantity Sold')
plt.title('Most populatr products')
plt.xticks(rotation=90)
plt.show()

###Further Analysis

In [ ]:
import matplotlib.pyplot as plt

# Plotting the histogram of the order_hour_of_day
plt.figure(figsize=(14, 7))
plt.hist(df['order_hour_of_day'], bins=24, color='skyblue', edgecolor='black')
plt.title('Distribution of Orders Throughout the Day')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Orders')
plt.xticks(range(0, 25))
plt.grid(axis='y', alpha=0.75)
plt.show()

# histogram_path


In [ ]:
# Calculate the count of orders per product
top_products = df['product_name'].value_counts().head(10)

# Plotting the bar chart for the top 10 most ordered products
plt.figure(figsize=(14, 7))
top_products.plot(kind='bar', color='lightcoral', edgecolor='black')
plt.title('Top 10 Most Ordered Products')
plt.xlabel('Product Name')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45, ha='right')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Setting the aesthetic style of the plots
sns.set_style("whitegrid")

# Create a figure to host the plots
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Plot 1: Distribution of Orders by Hour of Day
sns.histplot(df['order_hour_of_day'], bins=24, kde=False, ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Distribution of Orders by Hour of Day')
axes[0, 0].set_xlabel('Hour of Day')
axes[0, 0].set_ylabel('Order Count')

# Plot 2: Frequency of Reorders
sns.countplot(x='reordered', data=df, ax=axes[0, 1], palette='coolwarm')
axes[0, 1].set_title('Frequency of Reorders')
axes[0, 1].set_xlabel('Reordered')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_xticklabels(['No', 'Yes'])

# Plot 3: Number of Products Ordered from Each Department (Top 10)
top_departments = df['department'].value_counts().head(10)
sns.barplot(y=top_departments.index, x=top_departments.values, ax=axes[1, 0], palette='viridis')
axes[1, 0].set_title('Top 10 Departments by Number of Products Ordered')
axes[1, 0].set_xlabel('Number of Products Ordered')
axes[1, 0].set_ylabel('Department')

# Plot 4: Days Since Prior Order Distribution
sns.histplot(df['days_since_prior_order'], bins=30, kde=True, ax=axes[1, 1], color='salmon')
axes[1, 1].set_title('Distribution of Days Since Prior Order')
axes[1, 1].set_xlabel('Days Since Prior Order')
axes[1, 1].set_ylabel('Frequency')

# Adjust layout
plt.tight_layout()

# Show plots
plt.show()

Market Basket Analysis

In [ ]:
df.head()

In [ ]:
print(df.columns)

In [ ]:
data_column = df.iloc[:, 1]
data = list(data_column.apply(lambda x: x.split(',')))
data

In [ ]:
merged = df[df['product_id'].map(df['product_id'].value_counts()) > 13000]
merged.head()

In [ ]:
mbanalysis = merged.groupby(['order_id', 'product_name'])['order_id'].count().unstack().reset_index().fillna(0).set_index('order_id')
mbanalysis.head()